In [4]:
config_file = 'delineation/configs/dislocation_matching_disp_and_warp_and var_home.yml'
aug_config_file = 'delineation/configs/aug.yml'
import torch
from delineation.configs.defaults_segmentation import _C as cfg
from delineation.datasets import make_data_loader
from delineation.models import build_model_list
from delineation.utils import settings, cost_volume_helpers
from delineation.utils.settings import evaluate_results

from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

import yaml

cfg.merge_from_file(config_file)
with open(aug_config_file, 'r') as ymlfile:
    cfg_aug = yaml.load(aug_config_file)

_device = settings.initialize_cuda_and_logging(cfg)  # '_device' is GLOBAL VAR

train_loader, val_loader = make_data_loader(cfg, cfg_aug)
seg_model, model = build_model_list(cfg, True)

Using cuda
unet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/dislocations_segmentation/150_segmentor.tar' (epoch 150)
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_04/dislocations_matching_disp_warp_var_joint_10addaug/150_seg.tar' (epoch 150)
scmnet
=> loaded checkpoint '/home/dagnyt/EPFL/dislocations/models/models_2020_04/dislocations_matching_disp_warp_var_joint_10addaug/150_scmnet_light.tar' (epoch 150)


In [6]:
seg_model.eval()
model.eval()
total_test_loss = 0
pix1_err_m, pix3_err_m, pix5_err_m, epe_m, count = 0, 0, 0, 0, 0

for batch_idx, (l, r, lgt, rgt, dlgt, l_name) in (enumerate(val_loader)):
        indices = cost_volume_helpers.volume_indices(2 * cfg.TRAINING.MAXDISP, len(l),
                                                     cfg.TRAINING.HEIGHT, cfg.TRAINING.WIDTH, _device)

        with torch.no_grad():
            l, r, lgt, rgt, dlgt = l.to(_device), r.to(_device), lgt.to(_device), rgt.to(_device), dlgt.to(_device)

            l_seg, l_segmap = seg_model(l)
            r_seg, r_segmap = seg_model(r)
            if abs(dlgt.max()>32) or abs(dlgt.min()>32):
                continue
            print(l_name)
            dl_scores = model(l_segmap, r_segmap)
            
            dl_ = F.softmax(-dl_scores, 2)
            dl = torch.sum(dl_.mul(indices), 2) - cfg.TRAINING.MAXDISP
            for i in range(0, len(dlgt)):
                pix1_err, pix3_err, pix5_err, epe = evaluate_results(dlgt[i], dl[i], lgt[i])
                print(pix1_err, pix3_err, pix5_err, epe, dlgt.min(), dlgt.max())
                pix1_err_m += pix1_err
                pix3_err_m += pix3_err
                pix5_err_m += pix5_err
                epe_m += epe
                count += 1
            torch.cuda.empty_cache()
            
print('here')
values = pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count
print('Mean per dataset: {}, {}, {}, {}'.format(pix1_err_m / count, pix3_err_m / count, pix5_err_m / count, epe_m / count))

('2020_04_15_pair_0_1_SIFT_translation0001_LEFT.png',)
0.5171343891821802 0.11067889228489396 0.01940353801982032 1.3559731 tensor(-15., device='cuda:0') tensor(6., device='cuda:0')
('2020_04_15_pair_13_14_SIFT_translation0014_LEFT.png',)
0.6111244162375763 0.24104897617051851 0.1429768889953299 2.8495824 tensor(-1., device='cuda:0') tensor(30., device='cuda:0')
('2020_04_15_pair_14_15_SIFT_translation0015_LEFT.png',)
0.6356679319211461 0.16872780702828455 0.06801763193339047 1.9047856 tensor(-19., device='cuda:0') tensor(12., device='cuda:0')
('2020_04_15_pair_1_2_SIFT_translation0002_LEFT.png',)
0.5661670038624241 0.1690270369689167 0.07637483906566121 1.8249056 tensor(-21., device='cuda:0') tensor(12., device='cuda:0')
('2020_04_15_pair_3_4_SIFT_translation0003_LEFT.png',)
0.5360410830999066 0.12450980392156863 0.011764705882352941 1.4326398 tensor(-9., device='cuda:0') tensor(10., device='cuda:0')
('2020_04_15_pair_5_6_SIFT_translation0005_LEFT.png',)
0.43586675600650904 0.04532401